# Data Collecting
데이터 모으기, 또는 web scraping 실습.


## 사전 준비

In [ ]:
#https 오류 무시
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

import os
os.chdir('/content/drive/MyDrive/Google Office/Colab/Data Science/OV Class')

(위에 있는 건 신경쓰지 마세요)

일단 핵심 모듈들부터 임포트해야겠죠? 불러옵시다.

불러올 애들은


1. requests 모듈 (인터넷 접속해서 정보 따오는 모듈)
2. re 모듈 (Regex 를 다루게 해 주는 모듈)
3. 판다스 (기본)
4. beautifulsoap(정보 따온 전체 페이지에서 필요한 부분만 가져오는 친구)

4개를 불러올거에요.

numpy 는 당분간 필요하지 않답니다.

In [ ]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

## 데이터 세트로부터 데이터 불러오기


일단 requests로 인터넷 데이터를 직접 가져오는 대신, 
csv나 txt로 된 데이터 세트를 불러와서 데이터 프레임을 만드는 과정을 실습해볼거에요.

kaggle 에서 서울의 대기오염 농도 데이터를 다운 받은 후, 불러오도록 할게요.
(https://www.kaggle.com/jihyeseo/seoulairreport?select=SeoulHourlyAvgAirPollution.csv)

In [ ]:
df = pd.read_csv('SeoulHourlyAvgAirPollution.csv')
df.head()

,측정일시,측정소명,이산화질소농도(ppm),오존농도(ppm),일산화탄소농도(ppm),아황산가스(ppm),미세먼지(㎍/㎥),초미세먼지(㎍/㎥)
0,201711242300,강남구,0.038,0.004,0.4,0.005,16.0,10.0
1,201711242200,강남구,0.031,0.008,0.4,0.005,17.0,9.0
2,201711242100,강남구,0.025,0.012,0.4,0.005,18.0,11.0
3,201711242000,강남구,0.033,0.007,0.4,0.005,21.0,12.0
4,201711241900,강남구,0.033,0.008,0.4,0.005,20.0,10.0


와! csv를 불러 와서 데이터 프레임을 만드는 데 성공했어요! 

df.head() 명령어는 데이터프레임의 첫 5 행 (row) 을 출력하는 명령어로, 긴 데이터 프레임을 미리보기 하는데 도움이 된답니다.

이 데이터 셋에서 측정소명, 즉 '강남구' 와 같은 것은 정성적(qualitative) 한 데이터고, 

반면 농도, 미세먼지와 같은 것들은 정량적(quantitative)한 것이 되겠죠?

이제 데이터 프레임의 정보를 살펴보도록 할까요. df.info() 를 써 보죠.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4225 entries, 0 to 4224
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   측정일시          4225 non-null   int64  
 1   측정소명          4225 non-null   object 
 2   이산화질소농도(ppm)  4187 non-null   float64
 3   오존농도(ppm)     4183 non-null   float64
 4   일산화탄소농도(ppm)  4183 non-null   float64
 5   아황산가스(ppm)    4187 non-null   float64
 6   미세먼지(㎍/㎥)     4166 non-null   float64
 7   초미세먼지(㎍/㎥)    4161 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 264.2+ KB


어?! 4225 개의 데이터 셋은 모두 완벽해야 하는데, 미세먼지, 초미세먼지 등의 칼럼은 갯수가 적어요!

이러한 이슈 때문에 데이터를 불러올 때는 거의 대부분 '정리' 가 필요하답니다.

일단 이 데이터는 넘어가보죠.

## 인터넷에서 직접 데이터 긁어오기

보통 web scraping, web crawling 이라 하면 이걸 뜻합니다.

requests 로 페이지 전체의 데이터를 받은 다음, beautifulsoap 으로 필요한 부분만 정리할 거에요.

**⚠ 주의 ⚠**

많은 인터넷 사이트들은 자원의 낭비 방지로 인해서 크롤링을 방지하고 있어요. 

즉 모든 사이트에서 정보를 얻는 것이 가능한 것은 아니니, 모델을 만들기 전에 크롤링이 되는 지 확인하는 것이 먼저입니다.

그럼 인터넷 기사에서 데이터를 긁어 오는 것을 해 볼까요?

시험삼아 ytn 기사를 가지고 해 보죠.

requests 로 url을 불러오면 된답니다.

In [ ]:
url = 'https://www.ytn.co.kr/_ln/0104_202105211026078916'
response = requests.get(url)
html = response.text.strip()

In [ ]:
print(html[:2000])

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="ko" xml:lang="ko">
<head>
<title>[국제]도지코인 1달러? 머스크 또 장난질...美 당국에 투자자들 피해호소 | YTN</title>
<link rel="shortcut icon" href="/img/comm/favicon.ico" type="image/x-icon">
<link rel="icon" href="/img/comm/favicon.ico" type="image/x-icon">
<meta http-equiv="X-UA-Compatible" content="IE=edge" />
<meta http-equiv="content-type" content="text/html;charset=UTF-8"/>
<meta name="referrer" content="unsafe-url">
<meta name="viewport" content="width=device-width,initial-scale=1">
<meta name="format-detection" content="telephone=no">
<meta name="apple-mobile-web-app-capable" content="yes" />
<meta name="naver-site-verification" content="824969a675a69732787fac7267be435e7d7eaacd"/>
<meta property="fb:pages" content="237776059583039">
<meta name='title' content='[국제]도지코인 1달러? 머스크 또 장난질...美 당국에 투자자들 피해호소 | YTN'/>
<meta name='description' c

맙소사.

이게 외계인 언어인가요...?

여기서 우리가 쓸 만한 정보를 찾아내려면 Parser(https://ko.wikipedia.org/wiki/%EA%B5%AC%EB%AC%B8_%EB%B6%84%EC%84%9D) 를 거쳐서 정제시켜야 합니다.

마침 가장 사용하기 쉬운 parser인 Beautifulsoup 를 들여왔으니, 바로 활용해 보도록 해요.

In [ ]:
soup = BeautifulSoup(html, 'lxml')
results = soup.select('div.article')[0].text
print(results)






테슬라 최고경영자 일론 머스크가 도지코인 관련 이미지와 글을 다시 개인 트위터에 올려 한때 도지코인 가격이 15% 급등했다고 미국 경제 전문매체 마켓 인사이더 등이 보도했습니다.

머스크는 현지 시간 20일 '사이버 바이킹'이라는 문구의 아트 네온 밑에 도지코인 마스코트, 시바견이 그려진 1달러 지폐를 놓아둔 이미지를 자신의 트위터에 게재했습니다.

해당 이미지 위에는 "저 도지는 얼마인가"라는 글을 달아 도지코인 가격이 1달러까지 오를 수 있다고 유도한 게 아니냐는 추측을 낳습니다.

실제로 일부 투자자들로 보이는 누리꾼들은 호응하는 댓글을 달았고 도지코인 가격은 트윗이 올라간 뒤 4분 만에 15%가 치솟았습니다.

그러나 이후 상승 폭은 줄어들었고 머스크의 트윗에 비난 댓글을 달거나 증권거래위원회에 피해를 호소하는 투자자들이 잇따랐습니다.

증권거래위원회는 가상화폐 규제 권한이 없는데 일부 누리꾼들은 머스크가 가상화폐 시장뿐 아니라 증시에도 영향을 미쳤기 때문에 머스크를 조사해 체포하거나 벌금을 물려야 한다고 촉구한 것으로 알려졌습니다.

마켓 인사이더는 머스크의 이번 트윗 효과가 오래가지 않았다며 가상화폐가 얼마나 변동성이 심한지를 다시 보여준 사례였다고 전했습니다.

김정회 (junghkim@ytn.co.kr)[저작권자(c) YTN & YTN plus 무단전재 및 재배포 금지] 


와! 인터넷 기사의 내용만 깔끔하게 따 올 수 있어요~

여기서는 Beautifulsoap 의 자세한 문법, 특히 CSS Selector에 대해서는 자세히 다루지 않을거에요 (사실 저도 잘 몰라요).

웹에서 따오는 정보가 모두 정확한 건 아니니, 이런 게 있다~ 정도로 알아 두시면 될듯! 